In [1]:
import pandas as pd

df_2019 = pd.read_csv("datasets/netflix_titles_2019.csv")
df_2020 = pd.read_csv("datasets/netflix_titles_2020.csv")
df_2021 = pd.read_csv("datasets/netflix_titles_2021.csv")
df_imdb = pd.read_pickle("datasets/imdb.pkl")

In [2]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5837 non-null   int64 
 1   title         5837 non-null   object
 2   director      3936 non-null   object
 3   cast          5281 non-null   object
 4   country       5410 non-null   object
 5   date_added    5195 non-null   object
 6   release_year  5837 non-null   int64 
 7   rating        5827 non-null   object
 8   duration      5837 non-null   object
 9   listed_in     5837 non-null   object
 10  description   5837 non-null   object
 11  type          5837 non-null   object
dtypes: int64(2), object(10)
memory usage: 547.3+ KB


In [3]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630026 entries, 0 to 1253547
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   titleType      630026 non-null  object 
 1   primaryTitle   630026 non-null  object 
 2   originalTitle  630026 non-null  object 
 3   startYear      630026 non-null  object 
 4   averageRating  630026 non-null  float64
 5   numVotes       630026 non-null  int64  
 6   primaryName    502106 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 38.5+ MB


In [4]:
df_imdb['startYear'] = df_imdb['startYear'].astype(str)
df_imdb['startYear'] = df_imdb['startYear'].str.extract('(\d+)', expand = False)

In [5]:
df_2019['release_year'] = df_2019['release_year'].astype(str)

In [6]:
imdb_2019_pre = pd.merge(df_2019, df_imdb, how = 'left', left_on = ["title", 'release_year'], right_on = ["primaryTitle", "startYear"])

In [7]:
imdb_2019_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6074 entries, 0 to 6073
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        6074 non-null   int64  
 1   title          6074 non-null   object 
 2   director       4110 non-null   object 
 3   cast           5503 non-null   object 
 4   country        5638 non-null   object 
 5   date_added     5430 non-null   object 
 6   release_year   6074 non-null   object 
 7   rating         6064 non-null   object 
 8   duration       6074 non-null   object 
 9   listed_in      6074 non-null   object 
 10  description    6074 non-null   object 
 11  type           6074 non-null   object 
 12  titleType      3971 non-null   object 
 13  primaryTitle   3971 non-null   object 
 14  originalTitle  3971 non-null   object 
 15  startYear      3971 non-null   object 
 16  averageRating  3971 non-null   float64
 17  numVotes       3971 non-null   float64
 18  primaryN

In [8]:
import json

duplicates_2019 = json.loads(imdb_2019_pre['show_id'].value_counts().to_json())
duplicate_keys_2019 = [int(k) for k,v in duplicates_2019.items() if v > 1]

indices_2019 = []

for i in range(len(imdb_2019_pre)):
    if imdb_2019_pre['show_id'][i] in duplicate_keys_2019:
        indices_2019.append(i)

imdb_2019_2 = imdb_2019_pre.drop(i for i in indices_2019)
imdb_2019_3 = imdb_2019_pre.iloc[(i for i in indices_2019),:]
imdb_2019_4 = imdb_2019_3.drop(imdb_2019_3[imdb_2019_3['director'] != imdb_2019_3['primaryName']].index)

imdb_2019 = pd.concat([imdb_2019_2, imdb_2019_4], axis = 0).drop_duplicates()

duplicates_2019_check = json.loads(imdb_2019['show_id'].value_counts().to_json())
duplicate_keys_2019_check = [int(k) for k,v in duplicates_2019_check.items() if v > 1]

print(duplicate_keys_2019_check)

[81073387, 80103818]


In [9]:
imdb_2019[(imdb_2019['show_id'] == 81073387) | (imdb_2019['show_id'] == 80103818)]

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,type,titleType,primaryTitle,originalTitle,startYear,averageRating,numVotes,primaryName
1493,81073387,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-14,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,Movie,TV Show,A Fortunate Man,Lykke-Per,2018,7.1,677.0,Bille August
1494,81073387,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-14,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,Movie,Movie,A Fortunate Man,Lykke-Per,2018,7.2,6153.0,Bille August
5325,80103818,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies","With ""The Office"" long in the past, middling s...",Movie,Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.3,20353.0,Ricky Gervais
5326,80103818,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies","With ""The Office"" long in the past, middling s...",Movie,Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.6,275.0,Ricky Gervais


In [10]:
imdb_2019 = imdb_2019.drop(['titleType', 'primaryTitle', 'originalTitle', 'startYear', 'primaryName'], axis = 1)
imdb_2019 = imdb_2019.drop([1493,5326])
imdb_2019.dropna(subset = ["averageRating"], inplace = True)

In [11]:
imdb_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3648 entries, 0 to 6035
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        3648 non-null   int64  
 1   title          3648 non-null   object 
 2   director       2853 non-null   object 
 3   cast           3293 non-null   object 
 4   country        3512 non-null   object 
 5   date_added     3576 non-null   object 
 6   release_year   3648 non-null   object 
 7   rating         3642 non-null   object 
 8   duration       3648 non-null   object 
 9   listed_in      3648 non-null   object 
 10  description    3648 non-null   object 
 11  type           3648 non-null   object 
 12  averageRating  3648 non-null   float64
 13  numVotes       3648 non-null   float64
dtypes: float64(2), int64(1), object(11)
memory usage: 427.5+ KB


In [12]:
df_2020['release_year'] = df_2020['release_year'].astype(str)
df_2021['release_year'] = df_2021['release_year'].astype(str)

In [13]:
imdb_2020_pre = pd.merge(df_2020, df_imdb, how = 'left', left_on = ["title", 'release_year'], right_on = ["primaryTitle", "startYear"])
imdb_2021_pre = pd.merge(df_2021, df_imdb, how = 'left', left_on = ["title", 'release_year'], right_on = ["primaryTitle", "startYear"])

In [14]:
imdb_2020_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6490 entries, 0 to 6489
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        6490 non-null   int64  
 1   type           6490 non-null   object 
 2   title          6490 non-null   object 
 3   director       4451 non-null   object 
 4   cast           5905 non-null   object 
 5   country        6003 non-null   object 
 6   date_added     6479 non-null   object 
 7   release_year   6490 non-null   object 
 8   rating         6480 non-null   object 
 9   duration       6490 non-null   object 
 10  listed_in      6490 non-null   object 
 11  description    6490 non-null   object 
 12  titleType      4320 non-null   object 
 13  primaryTitle   4320 non-null   object 
 14  originalTitle  4320 non-null   object 
 15  startYear      4320 non-null   object 
 16  averageRating  4320 non-null   float64
 17  numVotes       4320 non-null   float64
 18  primaryN

In [15]:
duplicates_2020 = json.loads(imdb_2020_pre['show_id'].value_counts().to_json())
duplicate_keys_2020 = [int(k) for k,v in duplicates_2020.items() if v > 1]

indices_2020 = []

for i in range(len(imdb_2020_pre)):
    if imdb_2020_pre['show_id'][i] in duplicate_keys_2020:
        indices_2020.append(i)

imdb_2020_2 = imdb_2020_pre.drop(i for i in indices_2020)
imdb_2020_3 = imdb_2020_pre.iloc[(i for i in indices_2020),:]
imdb_2020_4 = imdb_2020_3.drop(imdb_2020_3[imdb_2020_3['director'] != imdb_2020_3['primaryName']].index)

imdb_2020 = pd.concat([imdb_2020_2, imdb_2020_4], axis = 0).drop_duplicates()

duplicates_2020_check = json.loads(imdb_2020['show_id'].value_counts().to_json())
duplicate_keys_2020_check = [int(k) for k,v in duplicates_2020_check.items() if v > 1]

print(duplicate_keys_2020_check)

[81073387, 80103818]


In [16]:
imdb_2020[(imdb_2020['show_id'] == 81073387) | (imdb_2020['show_id'] == 80103818)]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,titleType,primaryTitle,originalTitle,startYear,averageRating,numVotes,primaryName
4181,80103818,Movie,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies","With ""The Office"" long in the past, middling s...",Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.3,20353.0,Ricky Gervais
4182,80103818,Movie,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies","With ""The Office"" long in the past, middling s...",Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.6,275.0,Ricky Gervais
5578,81073387,Movie,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-14,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,TV Show,A Fortunate Man,Lykke-Per,2018,7.1,677.0,Bille August
5579,81073387,Movie,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-14,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,Movie,A Fortunate Man,Lykke-Per,2018,7.2,6153.0,Bille August


In [17]:
imdb_2020 = imdb_2020.drop(['titleType', 'primaryTitle', 'originalTitle', 'startYear', 'primaryName'], axis = 1)
imdb_2020 = imdb_2020.drop([4182,5578])
imdb_2020.dropna(subset = ["averageRating"], inplace = True)

imdb_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3974 entries, 0 to 5828
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        3974 non-null   int64  
 1   type           3974 non-null   object 
 2   title          3974 non-null   object 
 3   director       3141 non-null   object 
 4   cast           3613 non-null   object 
 5   country        3813 non-null   object 
 6   date_added     3973 non-null   object 
 7   release_year   3974 non-null   object 
 8   rating         3968 non-null   object 
 9   duration       3974 non-null   object 
 10  listed_in      3974 non-null   object 
 11  description    3974 non-null   object 
 12  averageRating  3974 non-null   float64
 13  numVotes       3974 non-null   float64
dtypes: float64(2), int64(1), object(11)
memory usage: 465.7+ KB


In [18]:
imdb_2021_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9147 entries, 0 to 9146
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        9147 non-null   object 
 1   type           9147 non-null   object 
 2   title          9147 non-null   object 
 3   director       6437 non-null   object 
 4   cast           8302 non-null   object 
 5   country        8294 non-null   object 
 6   date_added     9137 non-null   object 
 7   release_year   9147 non-null   object 
 8   rating         9143 non-null   object 
 9   duration       9144 non-null   object 
 10  listed_in      9147 non-null   object 
 11  description    9147 non-null   object 
 12  titleType      6107 non-null   object 
 13  primaryTitle   6107 non-null   object 
 14  originalTitle  6107 non-null   object 
 15  startYear      6107 non-null   object 
 16  averageRating  6107 non-null   float64
 17  numVotes       6107 non-null   float64
 18  primaryN

In [21]:
duplicates_2021 = json.loads(imdb_2021_pre['show_id'].value_counts().to_json())
duplicate_keys_2021 = [k for k,v in duplicates_2021.items() if v > 1]

indices_2021 = []

for i in range(len(imdb_2021_pre)):
    if imdb_2021_pre['show_id'][i] in duplicate_keys_2021:
        indices_2021.append(i)

imdb_2021_2 = imdb_2021_pre.drop(i for i in indices_2021)
imdb_2021_3 = imdb_2021_pre.iloc[(i for i in indices_2021),:]
imdb_2021_4 = imdb_2021_3.drop(imdb_2021_3[imdb_2021_3['director'] != imdb_2021_3['primaryName']].index)

imdb_2021 = pd.concat([imdb_2021_2, imdb_2021_4], axis = 0).drop_duplicates()

duplicates_2021_check = json.loads(imdb_2021['show_id'].value_counts().to_json())
duplicate_keys_2021_check = [k for k,v in duplicates_2021_check.items() if v > 1]

print(duplicate_keys_2021_check)

['s3895', 's5611']


In [22]:
imdb_2021[(imdb_2021['show_id'] == 's3895') | (imdb_2021['show_id'] == 's5611')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,titleType,primaryTitle,originalTitle,startYear,averageRating,numVotes,primaryName
4043,s3895,Movie,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-MA,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,TV Show,A Fortunate Man,Lykke-Per,2018,7.1,677.0,Bille August
4044,s3895,Movie,A Fortunate Man,Bille August,"Esben Smed, Katrine Rosenthal, Benjamin Kitter...",Denmark,"April 20, 2019",2018,TV-MA,168 min,"Dramas, International Movies",A gifted engineer flees his austere roots to p...,Movie,A Fortunate Man,Lykke-Per,2018,7.2,6153.0,Bille August
5845,s5611,Movie,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies, Music & Musicals","With ""The Office"" long in the past, middling s...",Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.3,20353.0,Ricky Gervais
5846,s5611,Movie,David Brent: Life on the Road,Ricky Gervais,"Ricky Gervais, Doc Brown, Tom Basden, Jo Hartl...","United Kingdom, United States","February 10, 2017",2016,TV-MA,97 min,"Comedies, International Movies, Music & Musicals","With ""The Office"" long in the past, middling s...",Movie,David Brent: Life on the Road,David Brent: Life on the Road,2016,6.6,275.0,Ricky Gervais


In [23]:
imdb_2021 = imdb_2021.drop(['titleType', 'primaryTitle', 'originalTitle', 'startYear', 'primaryName'], axis = 1)
imdb_2021 = imdb_2021.drop([4182,5578])
imdb_2021.dropna(subset = ["averageRating"], inplace = True)

imdb_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5662 entries, 0 to 9092
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        5662 non-null   object 
 1   type           5662 non-null   object 
 2   title          5662 non-null   object 
 3   director       4532 non-null   object 
 4   cast           5146 non-null   object 
 5   country        5370 non-null   object 
 6   date_added     5662 non-null   object 
 7   release_year   5662 non-null   object 
 8   rating         5660 non-null   object 
 9   duration       5659 non-null   object 
 10  listed_in      5662 non-null   object 
 11  description    5662 non-null   object 
 12  averageRating  5662 non-null   float64
 13  numVotes       5662 non-null   float64
dtypes: float64(2), object(12)
memory usage: 663.5+ KB


In [24]:
imdb_2019.to_pickle("datasets/imdb_2019.pkl")
imdb_2020.to_pickle("datasets/imdb_2020.pkl")
imdb_2021.to_pickle("datasets/imdb_2021.pkl")